In [45]:
import numpy as np
import pandas as pd

In [44]:
import scipy.stats as sts

In [4]:
df = pd.DataFrame({'item': ['manzana', 'sandia', 'pera', 'pizza', 'perrito', 'hamburguesa',
                            'ensalada', 'hummus', 'chuleton', 'longaniza', 'lubina'],
                   'a': [5, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0],
                   'b': [4, 3, 0, 5, 4, 4, 0, 0, 4, 4, 0],
                   'c': [4, 3, 3, 0, 0, 0, 3, 4, 2, 0, 2],
                   'd': [1, 2, 1, 5, 5, 5, 0, 2, 4, 3, 0],
                   'e': [5, 5, 5, 0, 0, 0, 4, 5, 0, 0, 0],
                   'f': [5, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0]})
df.set_index('item', inplace=True)
df


,a,b,c,d,e,f
item,,,,,,
manzana,5,4,4,1,5,5
sandia,5,3,3,2,5,5
pera,0,0,3,1,5,0
pizza,0,5,0,5,0,0
perrito,0,4,0,5,0,0
hamburguesa,0,4,0,5,0,0
ensalada,4,0,3,0,4,4
hummus,0,0,4,2,5,5
chuleton,0,4,2,4,0,0


 Binarizamos las valoraciones 

In [29]:
df_bin = (df>0).astype(int)
df_bin

,a,b,c,d,e,f
item,,,,,,
manzana,1,1,1,1,1,1
sandia,1,1,1,1,1,1
pera,0,0,1,1,1,0
pizza,0,1,0,1,0,0
perrito,0,1,0,1,0,0
hamburguesa,0,1,0,1,0,0
ensalada,1,0,1,0,1,1
hummus,0,0,1,1,1,1
chuleton,0,1,1,1,0,0


Teoria de conjuntos en Python

In [8]:
A = set([1,2,4])

In [9]:
B =set([2,4,6])

In [10]:
A.intersection(B)

{2, 4}

In [11]:
A.union(B)

{1, 2, 4, 6}

In [12]:
len(A.intersection(B))

2

Jaccard - similitud = (A inteseccion B / A Union B) - cuantas cosas tenemos en comun, si hemos visto las mismas cosas


0= No nos parecemos/ 1 = Somos casi iguales
Distancia - 0=cerca/ 1=lejos - es lo opuesto a similitud
Distancia = 1 - similitud

In [14]:
jaccard = lambda a,b: len(a.intersection(b))/ len(a.union(b))

In [15]:
jaccard(set([1,2,3]),set([1,3,4]))

0.5

In [16]:
jaccard(set([1,2,3]),set([1,2,3]))

1.0

In [17]:
jaccard(set([1,2,3]),set([4,5,6]))

0.0

In [30]:
set(df_bin[df_bin['a']>0].index)

{'ensalada', 'manzana', 'sandia'}

Vamos a encontrar las distancias desde el usuario A a todos los demas 

In [20]:
for u0 in df_bin.columns:
    for u1 in df_bin.columns:
        if u0 != u1:
            items0 = set(df_bin[df_bin[u0]>0].index)
            items1 = set(df_bin[df_bin[u1]>0].index)
            print('sim{}, {})) = {}'.format(u0,u1,jaccard(items0,items1)))


sima, b)) = 0.25
sima, c)) = 0.42857142857142855
sima, d)) = 0.2
sima, e)) = 0.6
sima, f)) = 0.75
simb, a)) = 0.25
simb, c)) = 0.2727272727272727
simb, d)) = 0.7777777777777778
simb, e)) = 0.2
simb, f)) = 0.2222222222222222
simc, a)) = 0.42857142857142855
simc, b)) = 0.2727272727272727
simc, d)) = 0.45454545454545453
simc, e)) = 0.7142857142857143
simc, f)) = 0.5714285714285714
simd, a)) = 0.2
simd, b)) = 0.7777777777777778
simd, c)) = 0.45454545454545453
simd, e)) = 0.4
simd, f)) = 0.3
sime, a)) = 0.6
sime, b)) = 0.2
sime, c)) = 0.7142857142857143
sime, d)) = 0.4
sime, f)) = 0.8
simf, a)) = 0.75
simf, b)) = 0.2222222222222222
simf, c)) = 0.5714285714285714
simf, d)) = 0.3
simf, e)) = 0.8


Recomendamos algo dando valor a cuanto me parezco a otro usuario - mejor que knn clusters 

Trade off between recommending what a lot of people likes vs recommending something original  - Youtube has done both, exploration and explotation in two different places

In [37]:
u0 = 'a'
neighbours = []

for u1 in df_bin.columns:
        if u0 != u1:
            items0 = set(df_bin[df_bin[u0]>0].index)
            items1 = set(df_bin[df_bin[u1]>0].index)
            neighbours.append((u1,jaccard(items0,items1)))


In [38]:
neighbours

[('b', 0.25), ('c', 0.42857142857142855), ('d', 0.2), ('e', 0.6), ('f', 0.75)]

In [39]:
top = sorted(neighbours, key= lambda x: -x[1])[:2]
top

[('f', 0.75), ('e', 0.6)]

In [40]:
#item -> score
recommendations = {}
for neighbour, similarity in top:
    for item in df_bin[df_bin[neighbour]>0].index:
        if item not in set(df_bin[df_bin['a']>0].index):
            recommendations[item] += similarity
        else:
            recommendations[item] = similarity
            

KeyError: 'hummus'

Deberia salir hummus y pera

*Cuando son ratings explicitos- usamos correlaciones con Pearson

In [ ]:
sts.pearsonr?

In [48]:
u0='a'
for u1 in df.columns:
    if u0 != u1:
        print( u0, u1, sts.pearsonr(df[u0],df[u1]))

a b (-0.014025268244765073, 0.9673537305875948)
a c (0.5630164352612619, 0.07133112524015639)
a d (-0.47773066916197693, 0.13724307333312027)
a e (0.6402913757205406, 0.033822514649123954)
a f (0.789823242704962, 0.0038279681529224444)


Nos interesan correlaciones positivas solo, negativas serian para recomendaciones antagonicas

Mirar Joblib para paralelizar cores

In [60]:
import io
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [61]:
data_root= "ml-100k/"

In [62]:
!cat ml-100k/README

cat: ml-100k/README: No such file or directory


In [63]:
columns = ['user_id','item_id','rating','timestamp']
datafile = os.path.join(data_root,'u.data')
data = pd.read_csv(datafile,sep='\t', names=columns)
data.head()

FileNotFoundError: File b'ml-100k/u.data' does not exist